In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import warnings

%matplotlib inline
warnings.filterwarnings('ignore')

In [ ]:
#load dataset
df_main = pd.read_csv('../input/cars-data/car data.csv')
df_main.head()

In [ ]:
df_main.info()

In [ ]:
df_main.describe()

In [ ]:
df_main['Age'] = 2022 - df_main['Year']
df_main.drop('Year',axis=1,inplace = True)

In [ ]:
df_main.rename(columns = {'Selling_Price':'Selling_Price(lacs)',
                          'Present_Price':'Present_Price(lacs)','Owner':'Past_Owners'},inplace = True)

# EDA

### correlation

univariate

In [ ]:
df_main.columns

In [ ]:
cat_cols = ['Fuel_Type','Seller_Type','Transmission','Past_Owners']
i=0
while i < 4:
    fig = plt.figure(figsize=[10,4])
    
    plt.subplot(1,2,1)
    sns.countplot(x=cat_cols[i], data=df_main)
    i += 1
    
    plt.subplot(1,2,2)
    sns.countplot(x=cat_cols[i], data=df_main)
    i += 1
    
    plt.show()

In [ ]:
num_cols = ['Selling_Price(lacs)','Present_Price(lacs)','Kms_Driven','Age']
i=0
while i < 4:
    fig = plt.figure(figsize=[13,3])
    plt.subplot(1,2,1)
    sns.boxplot(x=num_cols[i], data=df_main)
    i += 1
    
    plt.subplot(1,2,2)
    sns.boxplot(x=num_cols[i], data=df_main)
    i += 1
    
    plt.show()

In [ ]:
df_main[df_main['Present_Price(lacs)'] > df_main['Present_Price(lacs)'].quantile(0.99)]

In [ ]:
df_main[df_main['Selling_Price(lacs)'] > df_main['Selling_Price(lacs)'].quantile(0.99)]

In [ ]:
df_main[df_main['Kms_Driven'] > df_main['Kms_Driven'].quantile(0.99)]

Bi/Multivariate

In [ ]:
sns.heatmap(df_main.corr(), annot=True, cmap="RdBu")
plt.show()

In [ ]:
df_main.corr()['Selling_Price(lacs)']

In [ ]:
df_main.pivot_table(values='Selling_Price(lacs)', index = 'Seller_Type', columns= 'Fuel_Type')

In [ ]:
df_main.pivot_table(values='Selling_Price(lacs)', index = 'Seller_Type', columns= 'Transmission')

## working with dummies

In [ ]:
df_main.head()

In [ ]:
df_main.drop(labels='Car_Name',axis= 1, inplace = True)

In [ ]:
df_main = pd.get_dummies(data = df_main,drop_first=True) 

In [ ]:
df_main.head()

In [ ]:
# Separating target variable and its features
y = df_main['Selling_Price(lacs)']
X = df_main.drop('Selling_Price(lacs)',axis=1)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()

X_trainnxt = pd.DataFrame(sc.fit_transform(X_train))
X_testnxt = pd.DataFrame(sc.transform(X_test))

X_trainnxt.columns = X_train.columns.values
X_testnxt.columns = X_test.columns.values

X_trainnxt.index = X_train.index.values
X_testnxt.index = X_test.index.values

X_train = X_trainnxt
X_test = X_testnxt

In [ ]:
X_test.head()

In [ ]:
from sklearn.metrics import r2_score
from sklearn.model_selection import cross_val_score

In [ ]:
from sklearn.linear_model import LinearRegression

lr = LinearRegression()
lr.fit(X_train,y_train)

y_pred = lr.predict(X_test)
plt.scatter(y_test,y_pred)
plt.xlabel("Targets (y_test)", size = 18)
plt.ylabel("Predictions (y_pred)", size = 18)
plt.show()

R2_test = r2_score(y_test,y_pred)
R2_test

In [ ]:
from sklearn.linear_model import Ridge
from sklearn.model_selection import RandomizedSearchCV

rg = Ridge()
alpha = np.logspace(-3,3,num=14)

rg_rs = RandomizedSearchCV(estimator = rg, param_distributions = dict(alpha=alpha))
rg_rs.fit(X_train, y_train)


y_pred = rg_rs.predict(X_test)
plt.scatter(y_test,y_pred)
plt.xlabel("Targets (y_test)", size = 18)
plt.ylabel("Predictions (y_pred)", size = 18)
plt.show()

R2_test = r2_score(y_test,y_pred)
R2_test

In [ ]:
from sklearn.linear_model import Lasso
from sklearn.model_selection import RandomizedSearchCV

ls = Lasso()
alpha = np.logspace(-3,3,num=14)

ls_rs = RandomizedSearchCV(estimator = ls, param_distributions = dict(alpha=alpha))
ls_rs.fit(X_train, y_train)


y_pred = ls_rs.predict(X_test)
plt.scatter(y_test,y_pred)
plt.xlabel("Targets (y_test)", size = 18)
plt.ylabel("Predictions (y_pred)", size = 18)
plt.show()

R2_test = r2_score(y_test,y_pred)
R2_test

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV

rf = RandomForestRegressor()

n_estimators=list(range(500,1000,100))
max_depth=list(range(4,9,4))
min_samples_split=list(range(4,9,2))
min_samples_leaf=[1,2,5,7]
max_features=['auto','sqrt']

param_grid = {"n_estimators":n_estimators,
              "max_depth":max_depth,
              "min_samples_split":min_samples_split,
              "min_samples_leaf":min_samples_leaf,
              "max_features":max_features}

rf_rs = RandomizedSearchCV(estimator = rf, param_distributions = param_grid)
rf_rs.fit(X_train, y_train)


y_pred = rf_rs.predict(X_test)
plt.scatter(y_test,y_pred)
plt.xlabel("Targets (y_test)", size = 18)
plt.ylabel("Predictions (y_pred)", size = 18)
plt.show()

R2_test = r2_score(y_test,y_pred)
R2_test

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import RandomizedSearchCV

gb = GradientBoostingRegressor()

learning_rate = [0.001, 0.01, 0.1, 0.2]
n_estimators=list(range(500,1000,100))
max_depth=list(range(4,9,4))
min_samples_split=list(range(4,9,2))
min_samples_leaf=[1,2,5,7]
max_features=['auto','sqrt']

# Hyperparameters dict
param_grid = {"learning_rate":learning_rate,
              "n_estimators":n_estimators,
              "max_depth":max_depth,
              "min_samples_split":min_samples_split,
              "min_samples_leaf":min_samples_leaf,
              "max_features":max_features}

gb_rs = RandomizedSearchCV(estimator = gb, param_distributions = param_grid)
gb_rs.fit(X_train, y_train)


y_pred = gb_rs.predict(X_test)
plt.scatter(y_test,y_pred)
plt.xlabel("Targets (y_test)", size = 18)
plt.ylabel("Predictions (y_pred)", size = 18)
plt.show()

R2_test = r2_score(y_test,y_pred)
R2_test

In [ ]:
import pickle
file = open("predict_price.pkl", "wb") # opening a new file in write mode
pickle.dump(gb_rs, file) 